In [ ]:
from deepom.bionano_utils import MoleculeSelector

selector = MoleculeSelector()
selector.top_mol_num = 10
selector.select_molecules()

## Yevgeni's part 

In [ ]:
selector.selected[2].plot_bnx_item()

In [ ]:
from deepom.bionano_compare import BionanoCompare
 
compare = BionanoCompare()
compare.read_cmap()
compare.make_refs()
compare.refs

In [ ]:
from deepom.config import Config
from deepom.localizer import LocalizerModule
import monai

localizer_module = LocalizerModule()
localizer_module.checkpoint_search_dir = Config.CHECKPOINT_SEARCH_DIR
localizer_module.load_checkpoint = True
localizer_module.init_ensure_module()

In [ ]:
from matplotlib.pyplot import eventplot, imshow, figure, xlim
i=3
#good molecules: 0,7,6
figure(figsize=(30, 3))
image_input = selector.selected[i].bionano_image.segment_image[0]
target_width = localizer_module.image_channels
source_width = image_input.shape[0] // 2 + 1
image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]
imshow(image_input, aspect="auto", cmap="gray")
inference_item = localizer_module.inference_item(image_input)
eventplot([inference_item.loc_pred, selector.selected[i].locs / 375], colors=["b", "r"])
# xlim([0, 500])

In [ ]:
image_input.shape[0]

In [ ]:
inference_item.loc_pred

In [ ]:
selector.selected[i].bionano_image.segment_image

In [ ]:
from deepom.aligner import Aligner
from deepom.my_utils import overlap_percentage
import numpy as np

ref_id = selector.selected[i].xmap_item.ref_id
aligner_reg = Aligner()
aligner_reg.align_params = {}

aligner_rev = Aligner()
aligner_rev.align_params = {}

qry = inference_item.loc_pred * 335

inverted_qry = np.sort((inference_item.loc_pred[-1] - inference_item.loc_pred)*335) #not sure, need to make sure its correct

aligner_reg.make_alignment(qry=qry, ref=compare.refs[ref_id])
selector.selected[i].xmap_item.ref_lims, aligner_reg.alignment_ref[[0, -1]] # aligner.alignment_qry

In [ ]:
aligner_rev.make_alignment(qry=inverted_qry, ref=compare.refs[ref_id])
selector.selected[i].xmap_item.ref_lims, aligner_rev.alignment_ref[[0, -1]], # aligner.alignment_qry

In [ ]:
regular_overlap = overlap_percentage(selector.selected[i].xmap_item.ref_lims, aligner_reg.alignment_ref[[0, -1]])
reverse_overlap = overlap_percentage(selector.selected[i].xmap_item.ref_lims, aligner_rev.alignment_ref[[0, -1]])

aligner = aligner_reg if regular_overlap > reverse_overlap else aligner_rev

regular_overlap*100, reverse_overlap*100

In [ ]:
import numpy
from scipy.interpolate import interp1d
from deepom.my_utils import getAligner , get_scaler_from_aligner

In [ ]:
scaler = get_scaler_from_aligner(aligner)

z = aligner.alignment_ref
y = (z-z[0]) / (scaler*335)
y = y+aligner.alignment_qry[0] /335
x = aligner.alignment_qry/335
f = interp1d(x=y,y=x)

# a = numpy.zeros_like(aligner.alignment_ref)
# b = numpy.zeros_like(aligner.alignment_ref)

In [ ]:
# scaler = scaler = get_scaler_from_aligner(aligner_rev)

# z = aligner_rev.alignment_ref
# y = (z-z[0]) / (scaler*335)
# y = y+aligner_rev.alignment_qry[0] /335
# x = aligner_rev.alignment_qry/335
# f = interp1d(x=y,y=x)

# a = numpy.zeros_like(aligner.alignment_ref)
# b = numpy.zeros_like(aligner.alignment_ref)

# ref = compare.refs[ref_id]
# start, end = aligner_rev.alignment_ref[[0, -1]]
# start_index = np.argmax(ref >= start)
# end_index = np.argmin(ref <= end)
# ref_crop = ref[start_index:end_index]

In [ ]:
ref = compare.refs[ref_id]
start, end = aligner.alignment_ref[[0, -1]]
start_index = np.argmax(ref >= start)
end_index = np.argmin(ref <= end)
ref_crop = ref[start_index:end_index]


In [ ]:
z_hat = (ref_crop-ref_crop[0]) / (scaler*335)
z_hat = z_hat+x[0]
y_hat = f(z_hat)

In [ ]:
figure(figsize=(30, 3))
image_input = selector.selected[i].bionano_image.segment_image[0]
target_width = localizer_module.image_channels
source_width = image_input.shape[0] // 2 + 1
image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]
imshow(image_input, aspect="auto", cmap="gray")
eventplot([inference_item.loc_pred, x,y_hat], colors=["g", "b", "r"])
xlim([0, 800])

## Our Part

In [ ]:
from deepom.utils import Config
from deepom.bionano_compare import DataPrep
from deepom.bionano_utils import BNXItemCrop, BionanoFileData
from numpy.random import default_rng
from deepom.localizer import LocalizerModule
from deepom.aligner import Aligner
from deepom.my_utils import getAligner
from matplotlib.pyplot import eventplot, imshow, figure, xlim
from pandas import Series, DataFrame
import numpy as np
from tqdm import tqdm, tnrange , tqdm_notebook



In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from deepom.config import Config
from deepom.localizer import LocalizerModule
import monai


In [ ]:
crop_size_bp = 50  *1000
crop_size = int(crop_size_bp / Config.BIONANO_NOMINAL_SCALE)

In [ ]:
localizer_module = LocalizerModule()
localizer_module.checkpoint_search_dir = Config.CHECKPOINT_SEARCH_DIR
localizer_module.load_checkpoint = True
localizer_module.init_ensure_module()
localizer_module.rng = default_rng(seed=42)

In [ ]:
cmap_file_data = BionanoFileData()
cmap_file_data.file = Config.REF_CMAP_FILE
cmap_file_data.read_bionano_file()

refs = Series({
    ref_id: ref_df[ref_df["LabelChannel"] == 1]["Position"].values
    for ref_id, ref_df in cmap_file_data.file_df.groupby("CMapId")
    })

## Iterate over selected molecules, crop localize and align

In [ ]:
rng = default_rng(seed=42)
mol_ranks = dict.fromkeys(range(24),0)

for i, mol in tqdm_notebook(enumerate(selector.selected)):

    
    image_len = mol.bionano_image.segment_image.shape[-1]
    start = rng.integers(0,image_len - crop_size)
    stop = start + crop_size
    
    segment_image = mol.bionano_image.segment_image[...,start: stop]
    crop = BNXItemCrop(mol, (start, stop), i)
    crop.crop_size_pixels = crop_size
    crop.crop_size_bp = crop_size_bp
    crop.segment_image = segment_image
    
    # finished croping image. moving to locolaizer
    
    figure(figsize=(30, 3))
    image_input = crop.segment_image[0]
    target_width = localizer_module.image_channels
    source_width = image_input.shape[0] // 2 + 1
    image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]
    # imshow(image_input, aspect="auto", cmap="gray")
    inference_item = localizer_module.inference_item(image_input)
    # eventplot([inference_item.loc_pred, selector.selected[0].locs / 375], colors=["b", "r"])
    # xlim([0, 50])
    
    # run analyzer for each molecule, saving all results
    qry = inference_item.loc_pred * 335
    inverted_qry = np.sort((stop - inference_item.loc_pred)*335) #not sure, need to make sure its correct
   

    
    len_ref = range(1,len(refs)+1)
    scores = {}
    for j in tqdm_notebook(len_ref,desc = str(i+1)):
        
        aligner1 = getAligner()
        aligner1.make_alignment(qry=qry, ref=refs[j])
        score = aligner1.score
        
        aligner2 = getAligner()
        aligner2.make_alignment(qry=inverted_qry, ref=refs[j])
        
        scores[str(j)] = max(score , aligner2.score)
        scores[str(j)] = score

    scores = dict(sorted(scores.items(), key=lambda score: -score[1]))
    mol_rank = list(scores.keys()).index(str(mol.xmap_item.ref_id))
    mol_ranks[mol_rank] += 1 

    
    

In [ ]:
mol_ranks[0]/sum(mol_ranks)

In [ ]:
localizer_module.image_data.targets

In [ ]:
x = 2 + 3